#Assignment 11
* 1-The first task is using the function (learnt in the course) to create the lags of consumption for 24 hours and the lags of radiation between 3 to 6 hours.

* 2-Define new function, that receives the time stamp of every line and creates all the time-related features and add them to your dataset (*hour*, *sin(hour)*, *cos(hour)*, *dayOfWeek*).

* 3-Use another function to include the *weekendFlag*. So you should have created another function called weekendFlagCreator that receives the dayOfWeek and return weekend flag (**0: weekday**, **1: weekend**).

* 4-Finally, use a function to add all of the mentioned time-related features.

#Reading the created datafrom frame Week 10

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
DF_joinedPath = "/content/gdrive/MyDrive/Colab Notebooks/N-EETBS Assignments/Saved Files/Week10_DF_joined"
DF_joined = pd.read_csv(DF_joinedPath,sep=",",index_col=0)
DF_joined.head(3)

,Consumption,Temperature,Irradiance
2014-01-01 06:00:00,0.0,39.36,0.000000
2014-01-01 07:00:00,0.0,37.86,0.052800
2014-01-01 08:00:00,0.0,38.97,0.773583


Editting the temperature data, as they should be shifted 1 hour due to the time zone mistake:

In [4]:
DF_joined["Temperature"]=DF_joined["Temperature"].shift(-1)
DF_joined.head(3)

,Consumption,Temperature,Irradiance
2014-01-01 06:00:00,0.0,37.86,0.000000
2014-01-01 07:00:00,0.0,38.97,0.052800
2014-01-01 08:00:00,0.0,45.85,0.773583


#**1-Adding the lagged value columns**

##Function

In [5]:
def laggedColumnCreator(df,columnName,lagStart,lagEnd,lagInterval):
  for i in range(lagStart,lagEnd+1,lagInterval):
    newColumnName = columnName + " -" + str(i) + "h"
    df[newColumnName] = df[columnName].shift(i)
    df.dropna(inplace=True)
  return df

##Temperature lagged values

In [6]:
laggedColumnCreator(DF_joined,"Temperature",1,6,1)
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h
2014-01-02 03:00:00,0.0,40.63,0.0,45.27,48.82,51.75,53.09,49.05,53.57
2014-01-02 04:00:00,0.0,38.09,0.0,40.63,45.27,48.82,51.75,53.09,49.05
2014-01-02 05:00:00,0.0,36.82,0.0,38.09,40.63,45.27,48.82,51.75,53.09


##Consumption lagged values

In [7]:
laggedColumnCreator(DF_joined,"Consumption",24,24,1)
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h,Consumption -24h
2014-01-03 03:00:00,0.0,29.60,0.0,30.37,29.50,30.79,33.09,34.69,35.85,0.0
2014-01-03 04:00:00,0.0,26.88,0.0,29.60,30.37,29.50,30.79,33.09,34.69,0.0
2014-01-03 05:00:00,0.0,29.45,0.0,26.88,29.60,30.37,29.50,30.79,33.09,0.0


As the consumption values is 0 for the intial rows, we can choose a middle row, using the following order to check weather the added column is right or not:

In [8]:
if DF_joined.loc["2014-07-02 13:00:00","Consumption -24h"] == DF_joined.loc["2014-07-01 13:00:00","Consumption"]:
  print("The conumption lagged value works right!")
else:
  print("Somthing goes wrong with the consumption lagged values...check it!")

The conumption lagged value works right!


##Irradiance lagged values

In [9]:
laggedColumnCreator(DF_joined,"Irradiance",3,6,1)
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h,Consumption -24h,Irradiance -3h,Irradiance -4h,Irradiance -5h,Irradiance -6h
2014-01-03 21:00:00,0.0,44.78,0.0,44.90,44.34,45.56,50.72,53.41,55.53,0.0,0.0,0.3165,2.015917,3.396667
2014-01-03 22:00:00,0.0,43.42,0.0,44.78,44.90,44.34,45.56,50.72,53.41,0.0,0.0,0.0000,0.316500,2.015917
2014-01-03 23:00:00,0.0,43.36,0.0,43.42,44.78,44.90,44.34,45.56,50.72,0.0,0.0,0.0000,0.000000,0.316500


#**2-Adding time-related features columns**

Parsing time and dates: As we import the data frame from an excel file, we must again parse the dates to make Python, precept the first column as data and time.

In [10]:
DF_joined.index = pd.to_datetime(DF_joined.index)

##Function

In [11]:
def timeRelatedCreator(DataFrame):
  DataFrame_index = DataFrame.index
  DataFrame["Hour"] = DataFrame_index.hour
  DataFrame["Sin(hour)"] = np.sin(DataFrame_index.hour)
  DataFrame["Cos(hour)"] = np.cos(DataFrame_index.hour)
  DataFrame["Day of Week"] = DataFrame_index.dayofweek
  return DataFrame


##Added columns

In [15]:
timeRelatedCreator(DF_joined)
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h,Consumption -24h,Irradiance -3h,Irradiance -4h,Irradiance -5h,Irradiance -6h,Hour,Sin(hour),Cos(hour),Day of Week,Weekend Flag
2014-01-03 21:00:00,0.0,44.78,0.0,44.90,44.34,45.56,50.72,53.41,55.53,0.0,0.0,0.3165,2.015917,3.396667,21,0.836656,-0.547729,4,0
2014-01-03 22:00:00,0.0,43.42,0.0,44.78,44.90,44.34,45.56,50.72,53.41,0.0,0.0,0.0000,0.316500,2.015917,22,-0.008851,-0.999961,4,0
2014-01-03 23:00:00,0.0,43.36,0.0,43.42,44.78,44.90,44.34,45.56,50.72,0.0,0.0,0.0000,0.000000,0.316500,23,-0.846220,-0.532833,4,0


#**3-Adding weekend flag**

##Function

In [16]:
def weekendFlagCreator(DataFrame,dayOfWeekColumnName):
  DataFrame["Weekend Flag"] = 0
  DataFrame.loc[(DataFrame.loc[:,dayOfWeekColumnName] == 0),"Weekend Flag"] = 1
  return DataFrame

##Added column

In [17]:
weekendFlagCreator(DF_joined,"Day of Week")
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h,Consumption -24h,Irradiance -3h,Irradiance -4h,Irradiance -5h,Irradiance -6h,Hour,Sin(hour),Cos(hour),Day of Week,Weekend Flag
2014-01-03 21:00:00,0.0,44.78,0.0,44.90,44.34,45.56,50.72,53.41,55.53,0.0,0.0,0.3165,2.015917,3.396667,21,0.836656,-0.547729,4,0
2014-01-03 22:00:00,0.0,43.42,0.0,44.78,44.90,44.34,45.56,50.72,53.41,0.0,0.0,0.0000,0.316500,2.015917,22,-0.008851,-0.999961,4,0
2014-01-03 23:00:00,0.0,43.36,0.0,43.42,44.78,44.90,44.34,45.56,50.72,0.0,0.0,0.0000,0.000000,0.316500,23,-0.846220,-0.532833,4,0


Check if the defined function works properly:

In [18]:
DF_joined.loc["2014-01-27 07:00:00"]

Consumption          0.000000
Temperature         49.080000
Irradiance           0.094817
Temperature -1h     48.750000
Temperature -2h     48.030000
Temperature -3h     49.240000
Temperature -4h     45.940000
Temperature -5h     49.620000
Temperature -6h     49.460000
Consumption -24h     0.000000
Irradiance -3h       0.000000
Irradiance -4h       0.000000
Irradiance -5h       0.000000
Irradiance -6h       0.000000
Hour                 7.000000
Sin(hour)            0.656987
Cos(hour)            0.753902
Day of Week          0.000000
Weekend Flag         1.000000
Name: 2014-01-27 07:00:00, dtype: float64

#**4-A function for all time-related features**

In [19]:
def allTimeRelatedCreator(DataFrame):
  DataFrame_index = DataFrame.index
  DataFrame["Hour"] = DataFrame_index.hour
  DataFrame["Sin(hour)"] = np.sin(DataFrame_index.hour)
  DataFrame["Cos(hour)"] = np.cos(DataFrame_index.hour)
  DataFrame["Day of Week"] = DataFrame_index.dayofweek
  DataFrame["Weekend Flag"] = 0
  DataFrame.loc[(DataFrame.loc[:,"Day of Week"] == 0),"Weekend Flag"] = 1
  return DataFrame

In [20]:
allTimeRelatedCreator(DF_joined)
DF_joined.head(3)

,Consumption,Temperature,Irradiance,Temperature -1h,Temperature -2h,Temperature -3h,Temperature -4h,Temperature -5h,Temperature -6h,Consumption -24h,Irradiance -3h,Irradiance -4h,Irradiance -5h,Irradiance -6h,Hour,Sin(hour),Cos(hour),Day of Week,Weekend Flag
2014-01-03 21:00:00,0.0,44.78,0.0,44.90,44.34,45.56,50.72,53.41,55.53,0.0,0.0,0.3165,2.015917,3.396667,21,0.836656,-0.547729,4,0
2014-01-03 22:00:00,0.0,43.42,0.0,44.78,44.90,44.34,45.56,50.72,53.41,0.0,0.0,0.0000,0.316500,2.015917,22,-0.008851,-0.999961,4,0
2014-01-03 23:00:00,0.0,43.36,0.0,43.42,44.78,44.90,44.34,45.56,50.72,0.0,0.0,0.0000,0.000000,0.316500,23,-0.846220,-0.532833,4,0


Saving the new dataframe as excel file:

In [21]:
DF_joinedFileName = "Week11_DF_joined"
SaveFolderPath = "/content/gdrive/MyDrive/Colab Notebooks/N-EETBS Assignments/Saved Files"
DF_joinedFilePath = os.path.join(SaveFolderPath,DF_joinedFileName)
DF_joined.to_csv(DF_joinedFilePath)